In [7]:
import ee

In [8]:
import geemap

In [9]:
import collections
collections.Callable = collections.abc.Callable

In [10]:
ee.Initialize()

In [17]:
javascript_code = """
var s2 = ee.ImageCollection("COPERNICUS/S2");
var admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2");

var bangalore = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))
var geometry = bangalore.geometry()
Map.centerObject(geometry)

var filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
  .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
  .filter(ee.Filter.bounds(geometry))

var image = filtered.median(); 

// Exercise

// Calculate the Normalized Difference Built-Up Index (NDBI) for the image
// Hint: NDBI = (SWIR1 – NIR) / (SWIR1 + NIR)
// Visualize the built-up area using a 'red' palette

"""

In [18]:
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

import ee
import geemap
Map = geemap.Map()

s2 = ee.ImageCollection("COPERNICUS/S2")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

bangalore = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))
geometry = bangalore.geometry()
Map.centerObject(geometry)

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image = filtered.median()

# Exercise

# Calculate the Normalized Difference Built-Up Index (NDBI) for the image
# Hint: NDBI = (SWIR1 – NIR) / (SWIR1 + NIR)
# Visualize the built-up area using a 'red' palette
Map
